# Observable Estimation

There are many cases in Forest Benchmarking, and QCVV broadly, where we are interested in estimating the expectations of some set of Pauli observables at the end of some circuit (aka `pyquil.Program`). Additionally, we may be interested in running the same circuit after preparation of different starting states. The `observable_estimation.py` module is designed to facilitate such experiments. In it you will find

- the `ObservablesExperiment` `class` which is the high-level structure housing the several `ExperimentSetting`s you wish to run around its core (unchanging) `program`


- the `ExperimentSetting` `dataclass` which specifies the preparation `in_state` to prepare and `observable` to measure for a particular experimental run. The `in_state` is represented by a `TensorProductState` while the `observable` is a `pyquil.PauliTerm` 


- the function `estimate_observables()` which packages the steps to compose a list of `pyquil.Program`s comprising an `ObservableExperiment`, run each program on the provided quantum computer, and return the `ExperimentResult`s.


- the `ExperimentResult` `dataclass` that records the `mean` and associated `std_err` that was estimated for an `ExperimentSetting`, along with some other metadata.


In many cases this core functionality allows you to easily specify and run experiments to get the observable expectations of interest without explicitly dealing with shot data, state and measurement preparation programs, qc memory addressing, etc.

Later in this notebook we will discuss additional functionality that enables grouping compatible settings to run in parallel and 'calibrating' observable estimates, but for now let's dive into an example to build familiarity.

The procedure which likely leverages this abstraction most fruitfully is [quantum process tomography](https://en.wikipedia.org/wiki/Quantum_tomography#Quantum_process_tomography). If you aren't familiar with tomography at all, check out this [notebook](tomography_process.ipynb) for some background. The goal is to experimentally determine/characterize some unknown process running on our quantum computer (QC). Typically, we have some ideal operation in mind specified by a `pyquil.Program` that we want to run on the QC. Running process tomography will help us understand what the actual experimental implementation of this process is on our noisy QC. Since process tomography involves running our process on many different start states and measuring several different observables, an `ObservablesExperiment` should considerably facilitate implementation of the procedure.

## Create the experiment
First, we need to specify our process. For simplicity say we are interested in the QC's implementation of a bit flip on qubit 0.

In [ ]:
from pyquil import Program
from pyquil.gates import X
process = Program(X(0)) # bit flip on qubit 0

Now we need to specify our various `ExperimentSettings`. For now I'll make use of a helper in `forest.benchmarking.tomography.py` that does this for us.

In [ ]:
from forest.benchmarking.tomography import _pauli_process_tomo_settings
tomo_expt_settings = list(_pauli_process_tomo_settings(qubits = [0]))
print(tomo_expt_settings)

The first element of this list is displayed as `ExperimentSetting[X+_0→(1+0j)*X0]`. The symbols inside `ExperimentSetting` encode information about the preparation and measurement. They should be intepreted as `"prepare"→ "measure"`.


So the encoding of this particular setting preparation, `X+_0`, tells us that this setting will prepare the `+` eigenstate of `X`, i.e. the state $|+\rangle = \frac{1}{\sqrt 2}\left( |0\rangle +  |1 \rangle\right)$ on the qubit 0. Furthermore, after the program is run on this state, we will measure the observable $X$ on qubit 0. `(1+0j)` is just a multiplicative coefficient, in this case $1$. This coefficient will automatically be factored in to the estimates of the expectation and std_err for the corresponding observable output by `estimate_observables`.

Now we need to pair up our settings with our process program for the complete experiment description.

In [ ]:
from forest.benchmarking.observable_estimation import ObservablesExperiment
tomography_experiment = ObservablesExperiment(settings=tomo_expt_settings, program=process)
print(tomography_experiment)

The first line is our process program, i.e. `tomography_experiment.program`, and each subsequent indexed line is one of our settings. Now we can proceed to get estimates of expectations for the observable in each setting (given the preparation, of course). 

## Run the experiment

We'll need to get a quantum computer object to run our experiment on. We'll get both an ideal Quantum Virtual Machine (QVM) which simulates our program/circuit without any noise, as well as a noisy QVM with a built-in default noise model.

In [ ]:
from pyquil import get_qc
ideal_qc = get_qc('2q-qvm', noisy=False)
noisy_qc = get_qc('2q-qvm', noisy=True)

Let's predict some of the outcomes when we simulate our program on `ideal_qc` so that our process `X(0)` is implemented perfectly without noise. 

First take setting `0: X+_0→(1+0j)*X0`.
We prepare the plus eigenstate of $X$ after which we run our program `X(0)` which does nothing to this particular state. When we measure the $X$ observable on the plus eigenstate then we should get an expectation of exactly 1.

Now for setting `12: Z+_0→(1+0j)*X0`
we prepare the plus eigenstate of $Z$, a.k.a. the state $|0\rangle$ after which we perform our bit flip `X(0)` which sends the state to $|1\rangle$. When we measure $X$ on this state we expect our results to be mixed 50/50 between plus and minus outcomes, so our expectation should converge to 0 for a large number of shots. 

In [ ]:
from forest.benchmarking.observable_estimation import estimate_observables
results = list(estimate_observables(ideal_qc, tomography_experiment, num_shots=500))
for idx, result in enumerate(results):
    if idx == 0:
        print('\nWe expect the result to be 1.0 +- 0.0')
        print(result, '\n')
    elif idx == 12:
        print('\nWe expect the result to be around 0.0. Try increasing the num_shots to see if it converges.')
        print(result, '\n')
    else:
        print(result)

The first number after the setting is the estimate of the expectation for that observable along with the standard error.

If we perform the same experiment but simulate with the somewhat noisy QVM then we no longer expect the run of setting `0: X+_0→(1+0j)*X0` to yield an estiamte of `1.0` with certainty.

In [ ]:
# this time use the noisy_qc
noisy_results = list(estimate_observables(noisy_qc, tomography_experiment, num_shots=500))
for idx, result in enumerate(noisy_results):
    if idx == 0:
        print('\nWe expect the result to be less than 1.0 due to noise.')
        print(result, '\n')
    elif idx == 12:
        print('\nWe expect the result to be around 0.0, but it may be biased due to readout error.')
        print(result, '\n')
    else:
        print(result)

## Error mitigation

In the last example we saw the estimated expectations deviate from the ideal. In fact, if you set `num_shots` to some large number, say 500,000, then you'll notice that the list of results have expectations grouped around three distinct values, roughly `.95`, `-.82`, and `.064`. These values are what one would predict given the states measured are $|+\rangle$, $|-\rangle$, and some equal superposition, respectively, and given that the readout noise model has the default parameters $\Pr(\rm{measure } \ 0 \ | \ \rm{actually }\  0) = .975$ and $\Pr(\rm{measure } \ 1 \ | \ \rm{actually }\  1) = .911$. 

$$ \langle + \left| X \right| + \rangle = .975 - (1 - .975) = .95 $$
$$ \langle - \left| X \right| - \rangle = -.911 + (1 - .911) \approx -.82 $$
$$ \frac{1}{2}\left(\langle + \left| + \langle - \right|\right) X \left(\left| + \rangle + \right|- \rangle \right) = \frac{1}{2}(.975 + (1 - .911) - .911 - (1 - .975)) = .064 $$

Note that the parameterization of readout noise assumes that we can only measure in the computational (Z) basis. Indeed, the way we 'measure X' or any other non-Z observable is by first performing some pre-measurement rotation and then measuring in the computational basis (we do these rotations so that the plus eigenstate on a single qubit corresponds to 0). Thus, all of our observables are impacted similarly by readout error; some measurements may incur some additional observable-dependent error due to the noisy pre-measurement rotations.

### Symmetrizing readout

The fact that the noise is not symmetric, that is  $\Pr(\rm{measure } \ 0 \ | \ \rm{actually }\  0) \neq \Pr(\rm{measure } \ 1 \ | \ \rm{actually }\  1)$, complicates the impact of the noise. For a general state both parameters impact the resulting expectation. Instead of explicitly measuring each such parameter we instead opt to 'symmetirze' readout, which has the effect of averaging these parameters into a single 'symmetrized readout error'. For a single qubit this is achieved by measuring two 'symmetrized' programs for each original measurement. The first program is exactly equivalent to the original. The second program is nearly identical, except that we perform a quantum bit flip before measuring the qubit, and post measurement we classically flip the result. We then aggregate these results into one collection of 'symmetrized readout results'. We can directly calculate how this procedure affects the above calculations:
$$ \langle + \left| X \right| + \rangle_{\rm{symm}} = \left(\langle + \left| X \right| + \rangle - \langle - \left| X \right| - \rangle \right)/2 = \left([.975 - (1 - .975)] - [-.911 + (1 - .911)]\right)/2  = (.95 + .82) / 2 = .885 $$
$$ \langle - \left| X \right| - \rangle_{\rm{symm}} = \left(\langle - \left| X \right| - \rangle - \langle + \left| X \right| + \rangle \right)/2 = \left([-.911 + (1 - .911)] - [.975 - (1 - .975)]\right)/2  = (-.82 - .95) / 2 = -.885  $$
$$ \left(\frac{1}{2}\left(\langle + \left| + \langle - \right|\right) X \left(\left| + \rangle + \right|- \rangle \right)\right)_{\rm{symm}} = 0$$
Where the last example is perhaps a bit more subtle; the physical state is not changed by the symmetrizing quantum bit flip but nonetheless half of the aggregated results are classically flipped so the readout bias is nonetheless eliminated.

### Calibrating estimates of symmetrized observable expectations

We know that when running on a QPU we will likely have substantial readout error (which you can estimate [here](readout_fidelity.ipynb)), and this will scale all of our expectations in a predictable way. We have included a calibration procedure that can help mitigate the impact of symmetrized readout error by estimating and undoing this scaling. 

Specifically, after obtaining symmetrized estimates for expectations of some set of observables during an experiment you can pass on these results to `calibrate_observable_estimates` which, for each observable in the set, will essentially prepare each eigenstate to calculate the min and max expectation possbile under the noise. 

Here we again leverage readout symmetrization. We first create an experiment that prepares one particular positive eigenstate for the observable and measures its expectation. We then symmetrize this program (under default exhaustive symmetrization this essentially prepares all eigenstates) to get back a single estimate of the `calibration_expectation` which is also the scaling parameter which divides the `raw_expectation`. These calibration estimates will be recorded in a copy of the input `ExperimentResult`s with the experiment expectations appropriately scaled by the inverse `calibration_expectation`.

Let's use our results above as an example.

In [ ]:
from forest.benchmarking.observable_estimation import calibrate_observable_estimates, exhaustive_symmetrization
pre_cal_symm_results = list(estimate_observables(noisy_qc, tomography_experiment, num_shots=500, 
                                                 symmetrization_method=exhaustive_symmetrization))
post_cal_results = list(calibrate_observable_estimates(noisy_qc, pre_cal_symm_results, num_shots=10000))
for post_cal_res in post_cal_results:
    print(post_cal_res, ' with pre-calibrated expectation ', round(post_cal_res.raw_expectation,2))
    print('This was calibrated using a calibration estimate of ', round(post_cal_res.calibration_expectation, 2))
    print()

Note that statistical fluctuations may drive the magnitude of calibrated expectations above 1. 

## Estimate compatible observables from the same data

We can use fewer runs on a quantum computer to **speed up** estimation of some sets of observables. Consider two different settings on two qubits:
`Z+_0 * Z+_1→(1+0j)*Z0Z1`
and 
`Z+_0 * Z+_1→(1+0j)*Z0`. If we think about translating these settings into circuits (assuming they are part of the same `ObservablesExperiment`) then we note that the circuits will be identitcal even though our observables are different. Indeed ZZ and ZI commute, so we expect to be able to measure both observables simultaneously. In particular, each of these terms is diagonal in a 'Tensor Product Basis' i.e. a basis which is the tensor product of elements of single qubit bases (note we could throw in the term IZ too). In this case the basis is the computational basis, i.e. each vector in the sum 
$$ (|0\rangle + |1 \rangle)_0 \otimes (|0\rangle + |1 \rangle)_1 = |00\rangle + |01\rangle + |10\rangle + |11\rangle$$
Another example of observables which are compatible in this way would be $X0I1$, $I0Z1$, and $X0Z1$. Meanwhile, although $X0Z1$ and $Z0X1$ commute, they are not simultaneously diagonalized in a *Tensor Prodcut* Basis, so we don't consider grouping these terms (this might change in the future--feel encouraged to contribute this! :).

We provide a method `group_settings` which will automatically form groups of settings whose observables share a Tensor Product Basis and can be estimated from the same set of shot data. When you pass the resultant experiment with grouped settings into `estimate_observables` then each group of observables is estimated using `num_shots` many results. 

We'll group a 2 qubit tomography experiment to see how many experiment runs we save.

In [ ]:
from forest.benchmarking.observable_estimation import group_settings
from pyquil.gates import CNOT

two_q_tomo_expt = ObservablesExperiment(list(_pauli_process_tomo_settings(qubits=[0,1])), Program(CNOT(0,1)))
grouped_tomo_expt = group_settings(two_q_tomo_expt)

# print the pre-grouped experiment
print(two_q_tomo_expt)

print()
# print the grouped experiment. There are mutliple settings per line
print(grouped_tomo_expt)

The number of experimental runs has been reduced from 540 to 324! (where `num_shots` measurements will be repeated for each run). This grouped experiment can be passed to `estimate_observables` per usual.

This potential for grouping motivates the design choice to represent the `_settings` within an `ObservablesExperiment` as a nested `List[List[ExperimentSetting]]`, where the inner lists organize the groups of settings that will be estimated from a shared set of data. When creating an `ObservablesExperiment` we've provided the option of passing in a flat `List[ExperimentSetting]` which will be expanded internally into a `List` of lenght-1 `List[ExperimentSetting]`s.

Keep in mind that currently (3 July '19) the covariance between simultaneously estimated expectations is not recorded, so you may run into trouble if you want a variance of some function of these expectations.

## More on `ExperimentSetting`'s `in_state` and `observable`

`ExperimentSetting` is the `dataclass` that specifies a preparation and measurement to be paired with the `program` in an `ObservablesExperiment`. 

`in_state` itself has the type of a `TensorProductState` `dataclass` which also resides in `observable_estimation.py`. You can instantiate a single qubit Pauli eigenstate using methods such as `plusX(qubit)` or `minusY(qubit)`. Tensor products of such states can be built up by multiplying `TensorProductState`s. We also include [SIC](https://en.wikipedia.org/wiki/SIC-POVM) states, which help reduce the number of runs for process tomography. 

In [ ]:
from forest.benchmarking.observable_estimation import zeros_state, plusX, minusY, TensorProductState
# creating settings

# all zeros, i.e. all plusZ
typical_start_state = zeros_state(qubits = range(6))
print(typical_start_state, '\n')

# X+_2 * Y-_5
xy = plusX(2) * minusY(5)
print(xy, '\n')

# Z-_0 * Y+_3
zy = TensorProductState.from_str('Z-_0 * Y+_3')
print(zy, '\n')

# tensor states together
zyxy = zy * xy
print(zyxy, '\n')

# SIC states. You only need the 4 SIC states on a qubit for process tomography, instead of the 6 Pauli eigenstates.
from forest.benchmarking.observable_estimation import SIC0, SIC3
# SIC0_0 * SIC3_1
sic0_0 = SIC0(0)
sic3_1 = SIC3(1)
sic0sic3 = sic0_0 * sic3_1
print(sic0sic3)

In [ ]:
from pyquil.paulis import PauliTerm, sX, sZ, sY
# creating observables
zy_pauli = sZ(0) * sY(1)
xzzy = .5j * sX(1) * sZ(2) * sZ(4) * sY(5)

print(zy_pauli)
print(xzzy)
print(xzzy[0])
print(xzzy[5])
print(xzzy.coefficient)

In [ ]:
from forest.benchmarking.observable_estimation import ExperimentSetting
setting = ExperimentSetting(typical_start_state, xzzy)
print(setting)
# the -> separates the start state from the observable

obs_expt = ObservablesExperiment([setting], Program(X(0)))
print('\nIn an experiment with program X(0):\n', obs_expt)


## Estimating observables in parts

### Construct an `ObservablesExperiment` and turn it into a list of `Program`s

In [ ]:
from forest.benchmarking.observable_estimation import generate_experiment_programs
q = 0
# make ExperimentSettings for tomographing the plus state
expt_settings = [ExperimentSetting(plusX(q), pt) for pt in [sX(q), sY(q), sZ(q)]]
print('ExperimentSettings:\n==================\n',expt_settings,'\n')

# make an ObservablesExperiment. 
# We already specified the plus state as initial state, so do an empty (identity) program
obs_expt = ObservablesExperiment(expt_settings, program=Program())
print('ObservablesExperiment:\n======================\n',obs_expt,'\n')

# convert it to a list of programs and a list of qubits for each program
expt_progs, qubits = generate_experiment_programs(obs_expt)
print('Programs and Qubits:\n====================\n')
for prog, qs in zip(expt_progs, qubits):
    print(prog, qs, '\n')


## At this point we can run directly or first symmetrize

## 1) run programs directly

In [ ]:
from forest.benchmarking.observable_estimation import _measure_bitstrings
# this is a simple wrapper that adds measure instructions for each qubit in qubits and runs each program.
results = _measure_bitstrings(noisy_qc, expt_progs, qubits, num_shots=5)
for bits in results:
    print(bits)

## 2) First symmetrize, then run

A symmetrization method should return  
1. a list of programs 
2. a list of measurement qubits associated with each program
3. a list of bits/bools indicating which qubits where flipped before measurement for each program
4. a list of ints indicating the group of results to which each program's outputs should be re-assigned after correction

In [ ]:
from forest.benchmarking.observable_estimation import exhaustive_symmetrization, consolidate_symmetrization_outputs
symm_progs, symm_qs, flip_arrays, groups = exhaustive_symmetrization(expt_progs, qubits)
print('2 symmetrized programs for each original:\n======================\n')
for prog in symm_progs:
    print(prog)

# now these programs can be run as above
results = _measure_bitstrings(noisy_qc, symm_progs, symm_qs, num_shots=5)

# we now need to consolidate these results using the information in flip_arrays and groups 
results = consolidate_symmetrization_outputs(results, flip_arrays, groups)

print('After consolidating we have twice the number of (symmetrized) shots as above:\n======================\n')
# we now have twice the number of symmetrized shots
for bits in results:
    print(bits)

## Now we can translate our bitarray shots into ExperimentResults with expectation values

In [ ]:
from forest.benchmarking.observable_estimation import shots_to_obs_moments, ExperimentResult
import numpy as np
expt_results_as_obs = []
# we use the settings from the ObservableExperiment to label our results
for bitarray, meas_qs, settings in zip(results, qubits, obs_expt):
    # settings is a list of settings run simultaneously for a given program;
    # in our case, we just had one setting per program so this isn't strictly uncessary
    for setting in settings:
        observable = setting.observable # get the PauliTerm
        
        # Obtain statistics from result of experiment
        obs_mean, obs_var = shots_to_obs_moments(bitarray, meas_qs, observable)
        
        expt_results_as_obs.append(ExperimentResult(setting, obs_mean, std_err = np.sqrt(obs_var),
                                                    total_counts = len(bitarray)))

for res in expt_results_as_obs:
    print(res)

## All of the above comprises `estimate_observables`

In [ ]:
from forest.benchmarking.observable_estimation import estimate_observables

for res in estimate_observables(noisy_qc, obs_expt, num_shots=5, symmetrization_method = exhaustive_symmetrization):
    print(res)


Finally, we note that `calibrate_observable_estimates` can be decomposed in a similar using calls to `get_calibration_program`. The workflow is essentially the same as above.